<a href="https://colab.research.google.com/github/johanhoffman/methods-in-computational-science/blob/main/MICS_Direct_Methods_theoahfeldt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Theo Puranen Åhfeldt**

# **Abstract**

The objective of this report is to implement algorithms for solving or related to solving linear equations. The implementations are tested on randomly generated matrices. The testing indicates that the implementations are successful.

# **About the code**

This report is written by Theo Puranen Åhfeldt based on a template by Johan Hoffman.

In [1]:
# Copyright (C) 2020,2021 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Theo Puranen Åhfeldt (tahfeldt@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This file is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Set up environment**

To have access to the neccessary modules you have to run this cell.

In [2]:
# Load neccessary modules.
import numpy as np

# **Introduction**

The problem investigated in this report is the implementation of sparse matrix-vector product, QR factorization, a direct solver for linear equations, and a solver for the least squares problem.

# **Method**

Numpy arrays are used to represent vectors and matrices. The algorithms are implemented by their description in the course literature. Each implementation is tested using known features of the algorithms, such that QR-factorization produces an orthogonal and upper diagonal matrix. Since finite precision arithmetic is not perfect, tests do not check for equality but that the result is close enough to what is desired.

## Sparse matrix-vector product

We represent sparse matrices using CRS. A matrix $A$ is a three tuple $(val, col\_idx, row\_ptr)$, where $val$ is all non-zero elements in the matrix, $col\_idx$ contains the index of the column for each value in $val$ and $row\_ptr$ contains integers pointing at the value in $val$ that starts each row by index. Since Python starts indexing at 0, I do the same for the indices stored in $col\_idx$ and $row\_ptr$. To get the matrix-vector product of $A$ and $x$, for each row we go through all non-zero elements and calculate the product with the corresponding element in $x$ and sum them up to get the inner product of $x$ with that row.

In [3]:
def sparse_matrix_vector_product(A, x):
    val, col_idx, row_ptr = A
    rows = len(row_ptr) - 1
    b = np.zeros(rows)
    for r in range(rows):
        for i in range(row_ptr[r], row_ptr[r+1]):
            col = col_idx[i]
            b[r] += x[col] * val[i]
    return b

### Tests

In order to test the function, we want to compare it to normal matrix vector multiplication. To be able to do this thoroughly, we implement a function that converts a numpy array matrix to a CRS triple. The value $-1$ is used to represent a row with no non-zero elements.

In [4]:
def dense_to_sparse(A):
    n, m = A.shape
    val, col_idx, row_ptr = ([], [], [])
    for r in range(len(A)):
        row_ptr.append(-1)
        for c in range(len(A[r])):
            if A[r][c] != 0:
                if row_ptr[r] == -1:
                    row_ptr[r] = len(val)
                val.append(A[r][c])
                col_idx.append(c)
    row_ptr.append(len(val))
    return (val, col_idx, row_ptr)

We can now verify that our sparse matrix vector product is correct by comparing it to numpy's `matmul`.

In [5]:
def verify_sparse_matrix_vector_product(A, x):
    sparse = dense_to_sparse(A)
    assert np.linalg.norm(np.matmul(A, x) - sparse_matrix_vector_product(sparse, x)) < 1e-10

def test_sparse_matrix_vector_product(tests, rows, cols):
    for _ in range(tests):
        test_mat = np.matrix.round(np.random.random((rows, cols)),1)
        test_vec = np.matrix.round(np.random.random((cols,)),1)
        verify_sparse_matrix_vector_product(test_mat, test_vec)

## QR factorization

The QR factorization algorithm is implemented using the modified Gram-Schmidt iteration, where each new column of $A$ is iteratively projected to the orthogonal subspace of each column already added to $Q$.

In [6]:
def QR_factorization(A):
    rows, cols = A.shape
    assert rows == cols, "Non square matrix"
    Q = np.zeros(A.shape)
    R = np.zeros(A.shape)
    for j in range(cols):
        v = A[:,j] 
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], v)
            v = v - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(v)
        Q[:,j] = v/R[j,j]
    return Q, R

### Tests

The output is verified in three ways. Firstly, we check that $R$ is upper triangular using numpy's `triu`. Secondly, we check that $Q$ is orthogonal, by checking that $Q^T = Q^{-1} \Leftrightarrow QQ^T = I$. Lastly we check that $QR$ really is a factorization of $A$.

In [7]:
def verify_QR_factorization(A):
    Q, R = QR_factorization(A)
    rows = Q.shape[0]
    assert np.array_equal(R, np.triu(R)), "R is not upper triangular"
    assert np.linalg.norm(np.matmul(Q, Q.transpose()) - np.identity(rows)) < 1e-10, "Q is not orthogonal"
    assert np.linalg.norm(np.matmul(Q, R) - A) < 1e-10, "QR is not equal to A"

def test_QR_factorization(tests, rows):
    for _ in range(tests):
        test_mat = np.random.random((rows, rows))
        verify_QR_factorization(test_mat)

## Direct solver

To solve the linear equation $Ax = b$ we use QR-factorization. This gives us $QRx = b \Leftrightarrow Rx = Q^{-1}b = Q^Tb$, which we can solve using backwards substitution.

In [8]:
def backwards_substitution(U, b):
    rows, cols = U.shape
    assert rows == cols, "Non square matrix"
    x = np.zeros(cols)
    for i in range(rows - 1, -1, -1):
        s = 0
        for j in range(cols - 1, i, -1):
            s += x[j]*U[i,j]
        x[i] = (b[i] - s)/U[i,i]
    return x

def direct_solver(A, b):
    Q, R = QR_factorization(A)
    return backwards_substitution(R, np.matmul(Q.transpose(), b))

### Tests

To verify the output we can simply check that $Ax = b \Leftrightarrow ||Ax - b|| = 0$. Another test it to first generate $y$ and $A$, then calculating $b = Ay$ and checking that $x$ produced by our function is equal to $y$ (or that $||x - y|| = 0$).

In [9]:
def verify_direct_solver(A, b):
    x = direct_solver(A, b)
    assert np.linalg.norm(np.matmul(A, x) - b) < 1e-10
    
def verify_direct_solver_2(A, y):
    b = np.matmul(A, y)
    x = direct_solver(A, b)
    assert np.linalg.norm(x - y)
    
def test_direct_solver(tests, rows):
    for _ in range(tests):
        test_mat = np.random.random((rows, rows))
        test_vec = np.random.random((rows,))
        verify_direct_solver(test_mat, test_vec)
        verify_direct_solver_2(test_mat, test_vec)
        
test_direct_solver(100, 20)

## Least squares problem

The least squares problem can easily be solved by transforming it into a normal linear equation by multiplying both sides by $A^T$. This equation can then be solved using our direct solver.

In [10]:
def least_squares(A, b):
    return direct_solver(np.matmul(A.transpose(), A), np.matmul(A.transpose(), b))

### Tests

In [11]:
def verify_least_squares(A, b):
    x = least_squares(A, b)
    y = np.linalg.lstsq(A, b, rcond=None)[0]
    assert np.abs(np.linalg.norm(np.matmul(A, x) - b) - np.linalg.norm(np.matmul(A, x) - b)) < 1e-10
    
def test_least_squares(tests, rows, cols):
    for _ in range(tests):
        test_mat = np.random.random((rows, cols))
        test_vec = np.random.random((rows,))
        verify_least_squares(test_mat, test_vec)

# **Results**

The following code cell runs all test and shows that the implementions are successful:

In [12]:
test_sparse_matrix_vector_product(100, 30, 20)
test_QR_factorization(100, 30)
test_direct_solver(100, 30)
test_least_squares(100, 20, 30)
print("OK! completed all tests")

OK! completed all tests


# **Discussion**

The tests are comprehensive enough to give a clear indication that the implementations are correct.